# ETL Worfklow
---

## Setup


Importing my libraries of choice

In [1]:
import os
import pyspark.sql
import requests

Getting our SQL Server credientials ahead of time

In [2]:
db_user = os.environ.get('DB_USER')
db_password = os.environ.get('DB_PASSWORD')

Creating a spark session 

In [3]:
spark_sesh = pyspark.sql.SparkSession.builder.appName('Credit Card ETL').getOrCreate()

In [4]:
spark_sesh

## Extract 
---

I'm going to extract data from two datasets pertaining to the same bank, which are:


- The Credit Card Dataset
    - cdw_sapp_customer.JSON
    - cdw_sapp_branch.JSON 
    - cdw_sapp_credit.JSON

<br>

- The Bank Loan Application Dataset
    - Loan Application API Endpoint

📝 Notes: <br>
- The customer.JSON file and branch.JSON file contains information about the bank customer and bank branch, respectively. 
- The credit.JSON file contains information about credit card transactions
- The Loan Application is with respect to loans for purchasing homes, and includes information like whether or not the individuals were approved, gender, maritual status, and income.

Extracting the JSON Files with sparksession.read.load()


In [5]:
# JSON Files
branch_df = spark_sesh.read.load('../../data/credit_card_dataset/cdw_sapp_branch.json', format='json')  # 👀 don't forget the to specify the format
credit_df = spark_sesh.read.load('../../data/credit_card_dataset/cdw_sapp_credit.json', format='json')
customer_df = spark_sesh.read.load('../../data/credit_card_dataset/cdw_sapp_custmer.json', format='json')

Defining an API endpoint


In [6]:
LOAN_API_ENDPOINT = "https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json"


Creating a function to create a dataframe from an API Endpoint: <br>
 
The function takes an API Endpoint URL and the current live spark session, <br>
checks to see if the HTTP request is OK and returns a pyspark.sql DataFrame.

In [7]:
# API 

def api_check(api_endpoint: str, spark_session: pyspark.sql.SparkSession) -> pyspark.sql.DataFrame:  # param :type -> output-type
    api = requests.get(LOAN_API_ENDPOINT)
    print(f"HTTP Status Code: {api.status_code}")
    if api.status_code == 200:
        api_df = spark_session.createDataFrame(api.json())
        return api_df



In [8]:
loan_df = api_check(LOAN_API_ENDPOINT, spark_sesh)

HTTP Status Code: 200


Let's check our DataFrames and make sure they are intact and ready for transformation

In [9]:
dataframe_dict = {}

dataframe_dict['branch'] = branch_df  # assign val to dict
dataframe_dict['credit'] = credit_df
dataframe_dict['customer'] = customer_df
dataframe_dict['loan'] = loan_df

In [10]:
for name, dataframe in dataframe_dict.items():  # k:v in (k, v)
    print(name)
    dataframe.printSchema()

branch
root
 |-- BRANCH_CITY: string (nullable = true)
 |-- BRANCH_CODE: long (nullable = true)
 |-- BRANCH_NAME: string (nullable = true)
 |-- BRANCH_PHONE: string (nullable = true)
 |-- BRANCH_STATE: string (nullable = true)
 |-- BRANCH_STREET: string (nullable = true)
 |-- BRANCH_ZIP: long (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)

credit
root
 |-- BRANCH_CODE: long (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_SSN: long (nullable = true)
 |-- DAY: long (nullable = true)
 |-- MONTH: long (nullable = true)
 |-- TRANSACTION_ID: long (nullable = true)
 |-- TRANSACTION_TYPE: string (nullable = true)
 |-- TRANSACTION_VALUE: double (nullable = true)
 |-- YEAR: long (nullable = true)

customer
root
 |-- APT_NO: string (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_CITY: string (nullable = true)
 |-- CUST_COUNTRY: string (nullable = true)
 |-- CUST_EMAIL: string (nullable = true)
 |-- CUST_PHONE: long (nullable = tru

## Transform
---

We are given a mapping document, which tells us what kind of transformations we should make on the data before loading it onto the server. <br>
Let's take a look:

<div align='center'>
<h3>Tables</h3>
<h3>Customer Table</h3>
<img src='../../images/customer_mapping_doc.png' width=1200px>
<h3>Branch Table</h3>
<img src='../../images/branch_mapping_doc.png' width=1200px>
<h3>Credit-Card Table</h3>
<img src='../../images/creditcard_mapping_doc.png' width=1200px>
</div>

📝 Notes: <br>
We are not given any specifics for the loan information dataframe, so I'm not going to touch that dataframe or modify any data types. <br>

Furthermore, if you're following along, now would be a good time to split up your <br> views into several windows if you're using VS code or another similar IDE to <br> enhance your workflow, like this: <br>
<img src='../../images/workflow_example.png' width="1200px">

### Transforming the Customer Table 


Let's Start with Customer Table: <br>

📝Note: <br>
Just to make sure I'm following along and not making mistakes, <br>
Im using a small pandas dataframe to store the mapping document, so I can see what changes need to be made iterate through them, using pandas as a checklist of sorts.

This is completely optional, and not required at all, but I prefer it.

In [58]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # ensuring pandas wont truncate col


In [59]:
customer_map = pd.read_clipboard()
customer_map


,Source Column Names,Mapping Logic,Target Table,Target Field names,Target DataType,Description
0,SSN,Direct Move,CDW_SAPP_CUSTOMER,SSN,VARCHAR,Social Security Number of the customer (National ID)
1,FIRST_NAME,Convert the Name to Title Case,CDW_SAPP_CUSTOMER,FIRST_NAME,VARCHAR,First Name of the Customer
2,MIDDLE_NAME,Convert the middle name in lower case,CDW_SAPP_CUSTOMER,MIDDLE_NAME,VARCHAR,Middle Name of the customer
3,LAST_NAME,Convert the Last Name in Title Case,CDW_SAPP_CUSTOMER,LAST_NAME,VARCHAR,Last Name of the customer
4,CREDIT_CARD_NO,Direct_move,CDW_SAPP_CUSTOMER,Credit_card_no,VARCHAR,Credit card number of customer
5,"STREET_NAME,APT_NO","Concatenate Apartment no and Street name of customer's Residence with comma as a seperator (Street, Apartment)",CDW_SAPP_CUSTOMER,FULL_STREET_ADDRESS,VARCHAR,Apartment no and Street name of customer's Residence
6,CUST_CITY,Direct Move,CDW_SAPP_CUSTOMER,CUST_CITY,VARCHAR,Customer’s Current City
7,CUST_STATE,Direct Move,CDW_SAPP_CUSTOMER,CUST_STATE,VARCHAR,Customer’s State code
8,CUST_COUNTRY,Direct move,CDW_SAPP_CUSTOMER,CUST_COUNTRY,VARCHAR,Customer’s country code
9,CUST_ZIP,Direct move,CDW_SAPP_CUSTOMER,CUST_ZIP,VARCHAR,Zip code of Customer's Country


In [60]:
customer_map['Mapping Logic'] = customer_map['Mapping Logic'].apply(lambda x: str(x).lower())
customer_map

,Source Column Names,Mapping Logic,Target Table,Target Field names,Target DataType,Description
0,SSN,direct move,CDW_SAPP_CUSTOMER,SSN,VARCHAR,Social Security Number of the customer (National ID)
1,FIRST_NAME,convert the name to title case,CDW_SAPP_CUSTOMER,FIRST_NAME,VARCHAR,First Name of the Customer
2,MIDDLE_NAME,convert the middle name in lower case,CDW_SAPP_CUSTOMER,MIDDLE_NAME,VARCHAR,Middle Name of the customer
3,LAST_NAME,convert the last name in title case,CDW_SAPP_CUSTOMER,LAST_NAME,VARCHAR,Last Name of the customer
4,CREDIT_CARD_NO,direct_move,CDW_SAPP_CUSTOMER,Credit_card_no,VARCHAR,Credit card number of customer
5,"STREET_NAME,APT_NO","concatenate apartment no and street name of customer's residence with comma as a seperator (street, apartment)",CDW_SAPP_CUSTOMER,FULL_STREET_ADDRESS,VARCHAR,Apartment no and Street name of customer's Residence
6,CUST_CITY,direct move,CDW_SAPP_CUSTOMER,CUST_CITY,VARCHAR,Customer’s Current City
7,CUST_STATE,direct move,CDW_SAPP_CUSTOMER,CUST_STATE,VARCHAR,Customer’s State code
8,CUST_COUNTRY,direct move,CDW_SAPP_CUSTOMER,CUST_COUNTRY,VARCHAR,Customer’s country code
9,CUST_ZIP,direct move,CDW_SAPP_CUSTOMER,CUST_ZIP,VARCHAR,Zip code of Customer's Country


In [61]:
customer_map['Mapping Logic'] = customer_map['Mapping Logic'].replace('direct_move', 'direct move')

In [62]:
customer_map['Target Field names'] = customer_map['Target Field names'].apply(lambda x: str(x).upper())
customer_map.head(2)

,Source Column Names,Mapping Logic,Target Table,Target Field names,Target DataType,Description
0,SSN,direct move,CDW_SAPP_CUSTOMER,SSN,VARCHAR,Social Security Number of the customer (National ID)
1,FIRST_NAME,convert the name to title case,CDW_SAPP_CUSTOMER,FIRST_NAME,VARCHAR,First Name of the Customer


Now I can see exactly what needs to be transformed. 

In [63]:
customer_transform_guide = customer_map[(customer_map['Mapping Logic'] != 'direct move')]
customer_transform_guide

,Source Column Names,Mapping Logic,Target Table,Target Field names,Target DataType,Description
1,FIRST_NAME,convert the name to title case,CDW_SAPP_CUSTOMER,FIRST_NAME,VARCHAR,First Name of the Customer
2,MIDDLE_NAME,convert the middle name in lower case,CDW_SAPP_CUSTOMER,MIDDLE_NAME,VARCHAR,Middle Name of the customer
3,LAST_NAME,convert the last name in title case,CDW_SAPP_CUSTOMER,LAST_NAME,VARCHAR,Last Name of the customer
5,"STREET_NAME,APT_NO","concatenate apartment no and street name of customer's residence with comma as a seperator (street, apartment)",CDW_SAPP_CUSTOMER,FULL_STREET_ADDRESS,VARCHAR,Apartment no and Street name of customer's Residence
10,CUST_PHONE,change the format of phone number to (xxx)xxx-xxxx,CDW_SAPP_CUSTOMER,CUST_PHONE,VARCHAR,Contact Number of the customer


In [64]:
customer_transform_guide.set_index('Source Column Names', inplace=True)

In [65]:
customer_transform_guide.filter(like='NAME', axis=0)

,Mapping Logic,Target Table,Target Field names,Target DataType,Description
Source Column Names,,,,,
FIRST_NAME,convert the name to title case,CDW_SAPP_CUSTOMER,FIRST_NAME,VARCHAR,First Name of the Customer
MIDDLE_NAME,convert the middle name in lower case,CDW_SAPP_CUSTOMER,MIDDLE_NAME,VARCHAR,Middle Name of the customer
LAST_NAME,convert the last name in title case,CDW_SAPP_CUSTOMER,LAST_NAME,VARCHAR,Last Name of the customer
"STREET_NAME,APT_NO","concatenate apartment no and street name of customer's residence with comma as a seperator (street, apartment)",CDW_SAPP_CUSTOMER,FULL_STREET_ADDRESS,VARCHAR,Apartment no and Street name of customer's Residence


#### Transforming the name

In [22]:
customer_df_backup = customer_df


In [23]:
import pyspark.sql.functions as F

In [24]:
customer_df = customer_df.withColumns({'FIRST_NAME':F.initcap(customer_df['FIRST_NAME']), 'LAST_NAME':F.initcap(customer_df['LAST_NAME'])})

In [25]:
customer_df = customer_df.withColumn('MIDDLE_NAME', F.lower(customer_df['MIDDLE_NAME']))

Confirming Transformation is correct

In [26]:
customer_df['FIRST_NAME', 'LAST_NAME', 'MIDDLE_NAME'].show(2)

+----------+---------+-----------+
|FIRST_NAME|LAST_NAME|MIDDLE_NAME|
+----------+---------+-----------+
|      Alec|   Hooper|         wm|
|      Etta|   Holman|    brendan|
+----------+---------+-----------+
only showing top 2 rows



#### Transforming the Address

The easiest way to do this is to generate a SQL temp-table that is attached to the current spark session. <br>
I'm operating on the temp-table with SQL instead of Python here and its much easier and cleaner.

In [27]:
test = customer_df.createOrReplaceGlobalTempView('customer')

In [28]:
customer_df = spark_sesh.sql("""
SELECT
    *,
    CONCAT(STREET_NAME, ", ", APT_NO) AS FULL_STREET_ADDRESS
FROM
    global_temp.customer
""")

📝Notes: <br>
It might be *expensive* to concatenate the columns apt_no and street_name in one dataframe and then include them *again*<br>
by using the `select *` statement and dropping the extraneous columns, but with this current dataset the operations are pretty fast, and I'd rather save time using `*` and `.drop()` than including each and every column manually. <br>

Either way, we still have one more transformations left and pyspark's dataframes are immutable, so we generate a new copy of a dataframe every time.

In [31]:
customer_df[['APT_NO', 'STREET_NAME', 'FULL_STREET_ADDRESS']].show(2, truncate=False)

+------+-----------------+----------------------+
|APT_NO|STREET_NAME      |FULL_STREET_ADDRESS   |
+------+-----------------+----------------------+
|656   |Main Street North|Main Street North, 656|
|829   |Redwood Drive    |Redwood Drive, 829    |
+------+-----------------+----------------------+
only showing top 2 rows



In [32]:
customer_df = customer_df.drop('APT_NO', 'STREET_NAME')

The address is now concatenated and the other two columns are dropped.

In [33]:
customer_df[['FULL_STREET_ADDRESS']].show(2, truncate=False)

+----------------------+
|FULL_STREET_ADDRESS   |
+----------------------+
|Main Street North, 656|
|Redwood Drive, 829    |
+----------------------+
only showing top 2 rows



In [34]:
customer_df.printSchema()

root
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_CITY: string (nullable = true)
 |-- CUST_COUNTRY: string (nullable = true)
 |-- CUST_EMAIL: string (nullable = true)
 |-- CUST_PHONE: long (nullable = true)
 |-- CUST_STATE: string (nullable = true)
 |-- CUST_ZIP: string (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)
 |-- MIDDLE_NAME: string (nullable = true)
 |-- SSN: long (nullable = true)
 |-- FULL_STREET_ADDRESS: string (nullable = true)



#### Transforming the Phone number

In [66]:
customer_transform_guide.filter(like='PHONE', axis=0)

,Mapping Logic,Target Table,Target Field names,Target DataType,Description
Source Column Names,,,,,
CUST_PHONE,change the format of phone number to (xxx)xxx-xxxx,CDW_SAPP_CUSTOMER,CUST_PHONE,VARCHAR,Contact Number of the customer


In [50]:
customer_df[['CUST_PHONE']].show(2)

+----------+
|CUST_PHONE|
+----------+
|   1237818|
|   1238933|
+----------+
only showing top 2 rows



Here we are only required to add an artificial area code and the parenthesis and dash, so I'll do just that.


Using Pyspark.sql's  user-defined function this time:

In [51]:
phone_refactor = F.udf(lambda phone: "(555)" + str(phone)[:3] + "-" + str(phone)[3:])
# phone[0, 3) + phone[3:]
# note the slice includes the end of the string with [:]

In [52]:
customer_df = customer_df.withColumn('CUST_PHONE', phone_refactor(customer_df['CUST_PHONE']))

In [53]:
customer_df[['CUST_PHONE']].show(2)

+-------------+
|   CUST_PHONE|
+-------------+
|(555)123-7818|
|(555)123-8933|
+-------------+
only showing top 2 rows



Checking the types, you can see I transformed the phone number to a more-appropriate string datatype while modifying the phone number. <br>
(It was previously long int type)

In [54]:
customer_df.dtypes

[('CREDIT_CARD_NO', 'string'),
 ('CUST_CITY', 'string'),
 ('CUST_COUNTRY', 'string'),
 ('CUST_EMAIL', 'string'),
 ('CUST_PHONE', 'string'),
 ('CUST_STATE', 'string'),
 ('CUST_ZIP', 'string'),
 ('FIRST_NAME', 'string'),
 ('LAST_NAME', 'string'),
 ('LAST_UPDATED', 'string'),
 ('MIDDLE_NAME', 'string'),
 ('SSN', 'bigint'),
 ('FULL_STREET_ADDRESS', 'string')]

In [ ]:
customer_df = customer_df.withColumn('SSN', customer_df['SSN'].cast('string'))

In [76]:
customer_df[['LAST_UPDATED']].show(2, truncate=False)

+-----------------------------+
|LAST_UPDATED                 |
+-----------------------------+
|2018-04-21T12:49:02.000-04:00|
|2018-04-21T12:49:02.000-04:00|
+-----------------------------+
only showing top 2 rows



In [78]:
customer_df= customer_df.withColumn('LAST_UPDATED', customer_df['LAST_UPDATED'].cast('timestamp'))

In [79]:
customer_df[['LAST_UPDATED']].show(2, truncate=False)

+-------------------+
|LAST_UPDATED       |
+-------------------+
|2018-04-21 12:49:02|
|2018-04-21 12:49:02|
+-------------------+
only showing top 2 rows



#### Confirming Types for Customer One Last Time

Here I'm just comparing the expected vs actual datatypes and making sure they are at least somewhat similar  - pyspark.sql does not have all of the exact datatypes that most sql database engines have. 
<br><br>
Also, not to mention pyspark.sql datatypes don't really have the ability to set constraints on what goes into their columns. <br>
For example, I created a database where CUST_STATE is VARCHAR(2), and while VARCHAR(2) and VARCHAR(32) take up the exact <br>
same amount of disk space, VARCHAR(2) can potentially help save us from some data-entry headaches later down the line.

In [138]:
def compare_types(mapping_requirements: pd.DataFrame, column_names: str, dataframe: pyspark.sql.DataFrame) -> pd.DataFrame:
    """
    Mapping Requirements -> df
    Column Names -> Name of the column on the Mapping Requirements df that contains the column names 
    Dataframe -> actual dataframe we are transforming, must be pyspark.sql df (for now)
    """
    mapping_column_names = ", ".join(name for name in mapping_requirements.columns)
    pyspark_df_column_names = ", ".join(name[0] for name in dataframe.dtypes)
    types_df = pd.DataFrame(dataframe.dtypes)  # make df of d-types
    dataframe_var_name_col = 'Current Column Names'
    dataframe_var_name_type = 'Current Column Types'
    types_df.columns = [dataframe_var_name_col, dataframe_var_name_type]  # set column names
    mapping_df = pd.merge(mapping_requirements, types_df, left_on=column_names, right_on=dataframe_var_name_col, how='outer')  # pd merge
    # print(f"Mapping Document Total Columns: {len(mapping_requirements.columns)}")
    # print(f"Mapping Document Columns: {mapping_column_names}")
    # print("")
    # print(f"DataFrame Total Columns: {len(dataframe.dtypes)}")
    # print(f"DataFrame Column Names: {pyspark_df_column_names}")
    return mapping_df 

In [139]:
compare_types(customer_map, 'Target Field names', customer_df)\
    [['Source Column Names', 'Mapping Logic', 'Target Field names', 'Current Column Names', 'Target DataType', 'Current Column Types',]]

,Source Column Names,Mapping Logic,Target Field names,Current Column Names,Target DataType,Current Column Types
0,SSN,direct move,SSN,SSN,VARCHAR,string
1,FIRST_NAME,convert the name to title case,FIRST_NAME,FIRST_NAME,VARCHAR,string
2,MIDDLE_NAME,convert the middle name in lower case,MIDDLE_NAME,MIDDLE_NAME,VARCHAR,string
3,LAST_NAME,convert the last name in title case,LAST_NAME,LAST_NAME,VARCHAR,string
4,CREDIT_CARD_NO,direct move,CREDIT_CARD_NO,CREDIT_CARD_NO,VARCHAR,string
5,"STREET_NAME,APT_NO","concatenate apartment no and street name of customer's residence with comma as a seperator (street, apartment)",FULL_STREET_ADDRESS,FULL_STREET_ADDRESS,VARCHAR,string
6,CUST_CITY,direct move,CUST_CITY,CUST_CITY,VARCHAR,string
7,CUST_STATE,direct move,CUST_STATE,CUST_STATE,VARCHAR,string
8,CUST_COUNTRY,direct move,CUST_COUNTRY,CUST_COUNTRY,VARCHAR,string
9,CUST_ZIP,direct move,CUST_ZIP,CUST_ZIP,VARCHAR,string


### Transforming the Branch Table

Let's take a look at the branch table: